# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brandfort,-28.7001,26.4597,56.61,36.0,2.0,6.93,ZA,1717089706
1,1,idri,27.5000,13.2667,98.04,9.0,0.0,8.63,LY,1717089707
2,2,san andros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
3,3,tiksi,71.6872,128.8694,36.81,88.0,55.0,2.57,RU,1717089474
4,4,thinadhoo,0.5333,72.9333,84.31,81.0,83.0,19.46,MV,1717089709


In [12]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     599 non-null    int64  
 1   City        599 non-null    object 
 2   Lat         565 non-null    float64
 3   Long        565 non-null    float64
 4   Max Temp    565 non-null    float64
 5   Humidity    565 non-null    float64
 6   Cloudiness  565 non-null    float64
 7   Wind Speed  565 non-null    float64
 8   Country     560 non-null    object 
 9   Date        599 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 46.9+ KB


In [13]:
city_data_df.dropna(inplace=True)
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 560 entries, 0 to 598
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     560 non-null    int64  
 1   City        560 non-null    object 
 2   Lat         560 non-null    float64
 3   Long        560 non-null    float64
 4   Max Temp    560 non-null    float64
 5   Humidity    560 non-null    float64
 6   Cloudiness  560 non-null    float64
 7   Wind Speed  560 non-null    float64
 8   Country     560 non-null    object 
 9   Date        560 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 48.1+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    s = "Humidity",
    color = "Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Long,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_df = city_data_df.copy()
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] < 80) & (ideal_df["Max Temp"] > 60)].reset_index(drop=True)

# Display sample data
ideal_df

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,akonolinga,3.7667,12.2500,71.96,97.0,94.0,5.99,CM,1717089709
1,10,hamilton,39.1834,-84.5333,73.26,59.0,75.0,6.91,US,1717089532
2,11,sidi salim,31.2710,30.7867,71.01,54.0,19.0,8.99,EG,1717089713
3,15,avarua,-21.2078,-159.7750,78.85,89.0,75.0,3.44,CK,1717089440
4,16,jbail,34.1211,35.6481,69.08,80.0,20.0,6.17,LB,1717089716
...,...,...,...,...,...,...,...,...,...,...
251,583,tadine,-21.5500,167.8833,69.76,68.0,100.0,18.68,NC,1717090075
252,587,sant feliu de guixols,41.7833,3.0333,71.92,53.0,87.0,7.96,ES,1717089995
253,592,sayat,38.7839,63.8803,76.87,10.0,0.0,13.62,TM,1717090081
254,595,marsabit,2.3284,37.9899,71.62,52.0,77.0,11.86,KE,1717090083


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
cols = ["City", "Country", "Lat", "Long", "Humidity"]
hotel_df = ideal_df[cols].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = pd.Series(dtype='object')

# Display sample data
hotel_df

,City,Country,Lat,Long,Humidity,Hotel Name
0,akonolinga,CM,3.7667,12.2500,97.0,NaN
1,hamilton,US,39.1834,-84.5333,59.0,NaN
2,sidi salim,EG,31.2710,30.7867,54.0,NaN
3,avarua,CK,-21.2078,-159.7750,89.0,NaN
4,jbail,LB,34.1211,35.6481,80.0,NaN
...,...,...,...,...,...,...
251,tadine,NC,-21.5500,167.8833,68.0,NaN
252,sant feliu de guixols,ES,41.7833,3.0333,53.0,NaN
253,sayat,TM,38.7839,63.8803,10.0,NaN
254,marsabit,KE,2.3284,37.9899,52.0,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1

params = {"categories":categories,
        "limit":limit,
        "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Long"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    status_code = response.status_code
    
    if status_code == 200:
        name_address = response.json()
    else:
        name_address = {} # failed request
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
akonolinga - nearest hotel: Hôtel Le diamant de Mireille
hamilton - nearest hotel: North Vista Manor
sidi salim - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
jbail - nearest hotel: فندق كلف بوتيك
adamstown - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
st. john's - nearest hotel: Alt Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
albany - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
rio grande - nearest hotel: Hotel Vila Moura Executivo
mostaganem - nearest hotel: Hôtel Munatec
jamestown - nearest hotel: DoubleTree Jamestown
nova vicosa - nearest hotel: Pousada Pontal da Barra
richards bay - nearest hotel: SUN1 Richards Bay
thompson - nearest hotel: Thompson Inn
tshabong - nearest hotel: No hotel found
siracusa - nearest hotel: Hotel Panorama
olyokminsk - nearest hotel: Колос
tilcara - nearest hotel: Casa Colorada - Hotel de Montaña
sondrio - nearest hotel: Grand Hotel della

,City,Country,Lat,Long,Humidity,Hotel Name
0,akonolinga,CM,3.7667,12.2500,97.0,Hôtel Le diamant de Mireille
1,hamilton,US,39.1834,-84.5333,59.0,North Vista Manor
2,sidi salim,EG,31.2710,30.7867,54.0,No hotel found
3,avarua,CK,-21.2078,-159.7750,89.0,Paradise Inn
4,jbail,LB,34.1211,35.6481,80.0,فندق كلف بوتيك
...,...,...,...,...,...,...
251,tadine,NC,-21.5500,167.8833,68.0,Hôtel Nengoné
252,sant feliu de guixols,ES,41.7833,3.0333,53.0,Hotel del Mar
253,sayat,TM,38.7839,63.8803,10.0,No hotel found
254,marsabit,KE,2.3284,37.9899,52.0,Chicho Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    s = "Humidity",
    color = "Humidity",
    hover_cols = "all")
    
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Long,Lat]   (Humidity,index,City,Country,Hotel Name)